In [1]:
import numpy as np
import numpy.linalg as la

import pandas as pd
import nibabel as nib

import glob as g

In [2]:
# abs(sum(sum(t.*log(y)+ (1-t).*log(1-y)))/numel(t)) 

In [3]:
def matrix(fname):
    a = nib.load(fname)
    return np.asanyarray(a.dataobj)

def evaluate(a,seg):
    A = matrix(a)
    Seg = matrix(seg)
    val=list(set(Seg.reshape(-1)))
    S=[]
    for i in val:
        S.append(np.mean(A[Seg==i]))
    return np.array(S)

In [4]:
F = sorted(g.glob("*/*lq_seg*nii.gz"))
print(F)

['ANTs/SL_lq_seg.nii.gz', 'ANTs/SL_lq_seg_multilab.nii.gz', 'ANTs/t1w_lq_seg.nii.gz', 'ANTs/t1w_lq_seg_multilab.nii.gz', 'ANTs/walnut_lq_seg.nii.gz', 'ANTs/walnut_lq_seg_multilab.nii.gz', 'FSL/SL_lq_seg.nii.gz', 'FSL/t1w_lq_seg.nii.gz', 'FSL/walnut_lq_seg.nii.gz', 'frees/SL_lq_seg.nii.gz', 'frees/t1w_lq_seg.nii.gz', 'frees/walnut_lq_seg.nii.gz']


In [5]:
DF = []
for f in F:
    softw = f.split("/")[0]
    img = f.split("/")[-1].split("_")[0]
    interp =  f.split("_")[-1].replace(".nii.gz","")
    
    name = softw + "_" + img + "_" + interp
    print(name)
    
    lq = "data/" + img + "_lq.nii.gz"
    hq = "data/" + img + "_hq.nii.gz"
    hq_seg = "data/" + img + "_hq_seg.nii.gz"
    
    gold = evaluate(hq, hq_seg)
    
    res  = evaluate(lq,  f)
    #print(gold, res)
    err = la.norm(gold - res, 2)/la.norm(gold, 2)
    
    print(err)
    DF.append([name, err])

ANTs_SL_seg
0.11715572
ANTs_SL_multilab
0.11714269
ANTs_t1w_seg
0.0025167628
ANTs_t1w_multilab
0.0028706791
ANTs_walnut_seg
0.0002865289
ANTs_walnut_multilab
0.0009803354
FSL_SL_seg
0.10844781
FSL_t1w_seg
0.0032679942
FSL_walnut_seg
0.003828886
frees_SL_seg
0.11715572
frees_t1w_seg
0.0025167628
frees_walnut_seg
0.0002865289


In [6]:
df = pd.DataFrame(DF, columns = ["name", "error"])

In [7]:
df

,name,error
0,ANTs_SL_seg,0.117156
1,ANTs_SL_multilab,0.117143
2,ANTs_t1w_seg,0.002517
3,ANTs_t1w_multilab,0.002871
4,ANTs_walnut_seg,0.000287
5,ANTs_walnut_multilab,0.000980
6,FSL_SL_seg,0.108448
7,FSL_t1w_seg,0.003268
8,FSL_walnut_seg,0.003829
9,frees_SL_seg,0.117156


In [8]:
#print(df.to_latex(index=False)) 

In [9]:
df.to_excel("VOI_undersampling.xlsx")